Plot Fig. 2a

In [1]:
import sys
import os
import argparse
import numpy as np
import numpy.ma as ma
import pandas as pd
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib as mpl

from matplotlib.lines import Line2D
from matplotlib.patches import Circle
from matplotlib.colors import BoundaryNorm, Normalize

from CP4.utils.make_shapes import *
from CP4.utils.make_colorbar import MidpointNormalize
from CP4.make_composites.a1_make_var_field import *
from CP4.make_composites.b1_make_var_field_anomaly import load_composite_mean_ano_field
from CP4.make_composites.significance.b_compute_var_field_significance_merge import load_composite_significance_pvalues_merge
from CP4.plots.p_config import *

Done
Done
Done
Done
Done
Done
Done
Done
Done
Done
Done


In [2]:
ds='CP4A'
res=4
var_ref='twb'
var='twb'
window=6
y0=1997
y1=2006
months=[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
q_thresh=0.95
t_thresh=26.
min_hw_size=100.  # km2
max_hw_size=1000000.  # km2
sw=[4., 4.]
tw=[-72, 72]
samtime=[19, 19]
n_days=3
meth='cc3d'
cnty=26
length=25.  # km
pvalue=0.01
regs=['WSahel', 'CSahel', 'ESahel', 'Guinea', 'CAfr']
flabel='a'

In [3]:
years = np.arange(y0, y1+1, 1)
years_ = str(y0) + '-' + str(y1)
months_ = "-".join([str(m) for m in months])

res_ = str(res) + 'km'

swlat = sw[0]
swlon = sw[1]
sw_ = str(swlat) + 'x' + str(swlon)

space_scale = str(min_hw_size) + '-' + str(max_hw_size)
space_scale_ = str(int(min_hw_size)) + '-' + str(int(max_hw_size))

tw_before = tw[0]
tw_after = tw[1]
assert tw_before <= tw_after, "Incorrect number of time steps"
tw_ = str(tw_before) + '_to_' + str(tw_after)

regs_sahel = [reg for reg in regs if 'Sahel' in reg]

In [4]:
#~ Get data

out_data = []
out_hclim = []
out_ano = []
out_n_hhee = []

for reg in regs:
    coords = study_regions[reg]
    lat_range = coords[0]
    lon_range = coords[1]
    lat_min = lat_range[0]
    lat_max = lat_range[1]
    lon_min = lon_range[0]
    lon_max = lon_range[1]

    vardata = load_composite_var(ds, res, var_ref, var, y0, y1, months, t_thresh, q_thresh, n_days, window, sw, tw, lat_range, lon_range, min_hw_size, max_hw_size, meth, cnty)

    varhclim = load_composite_var_clim(ds, res, var_ref, var, y0, y1, months, t_thresh, q_thresh, n_days, window, sw, tw, lat_range, lon_range, min_hw_size, max_hw_size, meth, cnty)

    ds_ano = load_composite_mean_ano_field(ds, res, var_ref, var, y0, y1, months, lat_range, lon_range, t_thresh, q_thresh, n_days, window, sw, tw, samtime, min_hw_size, max_hw_size, meth, cnty)

    out_data.append(vardata)
    out_hclim.append(varhclim)
    out_ano.append(ds_ano)
    out_n_hhee.append(pd.Series(ds_ano.shape[0]))

In [ ]:
#~ Treat data

out_data = np.concatenate(out_data, axis=0)
out_hclim = np.concatenate(out_hclim, axis=0)
out_ano = xr.concat(out_ano, dim='n')
out_n_hhee = pd.concat(out_n_hhee, axis=1)

out_n_hhee.columns = regs
out_n_hhee['Sahel'] = out_n_hhee[regs_sahel].sum(axis=1)

out_n_hhee.drop(regs_sahel, axis=1, inplace=True)
if regs == ['WSahel', 'CSahel', 'ESahel', 'Guinea', 'CAfr']:
    out_n_hhee = out_n_hhee[['Sahel', 'Guinea', 'CAfr']]


n_hhee = out_hclim.shape[0]

tw_before = tw[1]+samtime[0]
tw_after = tw[1]+samtime[1]

out_hclim_time = out_hclim[:,tw_before:tw_after+1,:,:].mean(axis=1)

out_hclim_mean = np.nanmean(out_hclim_time, axis=0) # mean across events
out_hclim_med = np.nanmedian(out_hclim_time, axis=0) # median across events

var_hhee_ano_mean = np.nanmean(out_ano.values, axis=0) # mean across events
var_hhee_ano_med = np.nanmedian(out_ano.values, axis=0) # median across events

out_hclim_2plot = out_hclim_mean
var_hhee_ano_2plot = var_hhee_ano_mean

out_hclimmean = np.nanmean(out_hclim_2plot)  # spatial mean
out_hclimmed = np.nanmedian(out_hclim_2plot)
varanomean = np.nanmean(var_hhee_ano_2plot)  # spatial mean
varanomed = np.nanmedian(var_hhee_ano_2plot)

xscirc = xsm[circ_mask]
yscirc = ysm[circ_mask]

vals_clim_disc = ma.masked_array(out_hclim_time, ~disc_mask)    # mask: True = masked
vals_clim_disc = ma.filled(vals_clim_disc, fill_value=np.nan)
vals_clim_disc_med = np.nanmean(vals_clim_disc, axis=0) # median across events
vals_clim_disc_mean = np.nanmean(vals_clim_disc_med)  # spatial mean

vals_ano_disc = ma.masked_array(out_ano.values, ~disc_mask)
vals_ano_disc = ma.filled(vals_ano_disc, fill_value=np.nan)
vals_ano_disc_med = np.nanmean(vals_ano_disc, axis=0) # median across events
vals_ano_disc_mean = np.nanmean(vals_ano_disc_med)  # spatial mean

pvalues_ = pvalues.where(pvalues > pval, np.nan)
pvalues_.values[~np.isnan(pvalues_)] = 1

In [ ]:
#~ Plot

